In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")


train_num = train.select_dtypes("int")
test_num = test.select_dtypes("int")
X = train_num.drop(columns="fav_novel_cnt_bin")
y = train_num[["fav_novel_cnt_bin"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
!pip install -q catboost

ERROR: Could not find a version that satisfies the requirement catboost (from versions: none)
ERROR: No matching distribution found for catboost


In [18]:
import numpy as np
X = pd.read_pickle("../exp_12/data/train.pkl")
categorical_features_indices = np.where((X.dtypes != np.float32) & (X.dtypes != np.float64))[0]
categorical_features_indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14, 195, 199, 203, 204, 205, 206, 207, 208])

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22170 entries, 17830 to 39999
Columns: 209 entries, userid to genre_ce
dtypes: float32(180), float64(6), int64(23)
memory usage: 20.3 MB


In [ ]:
from catboost import Pool
import numpy as np

# カテゴリのカラムのみを抽出
categorical_features_indices = np.where(X.dtypes != np.float)[0]

# データセットの作成。Poolで説明変数、目的変数、
# カラムのデータ型を指定できる
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

from catboost import CatBoostClassifier

params = {
    'loss_function':'MultiClass'
    'depth' : 6,                  # 木の深さ
    'learning_rate' : 0.05,       # 学習率
    'early_stopping_rounds' : 10,
    'iterations' : 200, 
    'custom_loss' :['Accuracy'], 
    'random_seed' :42
}
# パラメータを指定した場合は、以下のようにインスタンスに適用させる
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)